In [54]:
import numpy as np

This is a practical example following my research on linear regression found at https://jonnys-world.com/data-science/linear-regression

# Linear Regression

Suppose we have the following contrived data where each row is the measure of the weight, count and price of a different pack of Chocolate from around the UK

<table>
  <tr>
    <th>Weight(kg)</th>
    <th>Sugar(g)</th>
    <th>Price (pence)</th>
  </tr>
  <tr>
    <td>100</td>
    <td>2</td>
    <td>5</td>
  </tr>
  <tr>
    <td>50</td>
    <td>42</td>
    <td>25</td>
  </tr>
    <tr>
    <td>45</td>
    <td>31</td>
    <td>22</td>
  </tr>
  <tr>
    <td>60</td>
    <td>35</td>
    <td>18</td>
  </tr>
</table>

Let's say we wanted to predict how much a pack of chocolate would be based on their height and sugar content. How could we do it?

Let's assume there is a linear relationship between the response and predictors. Now we can assume a linear relationship between predictors (weight and sugar) and the response (price of fredos). We can represent this relationship using the following equation $ \hat Y = X^T \beta $

In [55]:
y = np.matrix([5,25,22,18]).transpose()

In [56]:
x = np.matrix([
    [1, 100, 2],
    [1, 50, 42],
    [1, 45, 31],
    [1, 60, 35]
])

(If you want to know where the 1's come from read my page)

## Calculating Beta

In order to make a prediction for any X we need to figure out what $ \beta $ is. We can figure that out using the normal equation $ \beta = (X^TX)^{-1}X^Ty $

In [57]:
beta = np.linalg.inv(x.transpose() * x) * x.transpose() * y

## Making a Prediction

Now let's say we wanted to predict the price of a bar of fredo with weight 120kg and sugar content 50g. We would plug it into the equation like so

In [58]:
new_value = np.matrix([[1, 120,50]])

In [59]:
prediction =  new_value * beta
prediction

matrix([[10.53421014]])

Hence our predict the chocolate bar will be 10 pence

## How do we know if any of our predictors are related to Y?

In the real world we normally split our data into a test set and a training set. We use the test set to train the model (as we did above). We use the test set to check how accurate our model is. <br> Let's assume that the i hid some data from you! The following table is the complete set of data. The last two rows are the bits of information which was witheld.

<table>
  <tr>
    <th>Weight(kg)</th>
    <th>Sugar(g)</th>
    <th>Price (pence)</th>
  </tr>
  <tr>
    <td>100</td>
    <td>2</td>
    <td>5</td>
  </tr>
  <tr>
    <td>50</td>
    <td>42</td>
    <td>25</td>
  </tr>
    <tr>
    <td>45</td>
    <td>31</td>
    <td>22</td>
  </tr>
  <tr>
    <td>60</td>
    <td>35</td>
    <td>18</td>
  </tr>
   <tr>
    <td>20</td>
    <td>1</td>
    <td>1</td>
  </tr>
  <tr>
    <td>30</td>
    <td>35</td>
    <td>20</td>
  </tr>
</table>

In [60]:
x_test = np.matrix([
    [20, 1, 1],
    [30, 35, 20]
])

We can use a formula called the RSS to tell us the amount of variance which is not explained by the regression

In [61]:
rss = np.square(5 - [1, 100, 2] * beta ) 
+ np.square(25 - [1, 50, 42] * beta )
+ np.square(22 - [1, 45, 31] * beta ) 
+ np.square(18 - [1, 60, 35] * beta )
+ np.square(1 - [1, 20, 1] * beta ) 
+ np.square(20 - [1, 30, 35] * beta)
rss

matrix([[0.12753845]])

We can use a formula called TSS to tell us the total amount of variance in the data set before regression

In [62]:
tss = np.square(5 - y.mean()) 
+ np.square(25 - y.mean()) 
+ np.square(22 - y.mean()) 
+ np.square(18 - y.mean())
+ np.square(2 - y.mean()) 
+ np.square(20 - y.mean())
tss

156.25

We can therefore calculate the variance explained by the regression. (TSS - RSS)

In [63]:
tss - rss

matrix([[156.12246155]])

Yey, a lot of the variance is explained by the regression!

To check if all of the regression coefficients are zero we test the null hypothesis <br>
$ H_0 : \beta_1 = \beta_2 = 0 $ <br>
versus the null hypothesis <br>
$ H_1 : $ at least one $ \beta_1 $ or $ \beta_2 $ is none zero <br>
we use the F statistic to test this hypothesis

In [64]:
p = x.shape[1]-1
N = x.shape[0]

In [65]:
F = ((tss - rss) / p ) / ( rss / ( N - p - 1))
F

matrix([[612.06034631]])

As our F statistic is significantly bigger than 1 we can reject our null hypothesis

Now we know at least one of our predictors has a relationship to our response!

## What if we want to test if a particular subset of our predictors relates to our response?

We fit another model which uses these predictors. Let's say we wanted to check if weight is related

In [70]:
y_weight = np.matrix([5,25,22,18]).transpose()
x_weight = np.matrix([
    [1, 100],
    [1, 50],
    [1, 45],
    [1, 6]
])
beta_weight = np.linalg.inv(x_weight.transpose() * x_weight) * x_weight.transpose() * y_weight
rss_weight = np.square(5 - [1, 100] * beta_weight ) 
+ np.square(25 - [1, 50] * beta_weight )
+ np.square(22 - [1, 45] * beta_weight ) 
+ np.square(18 - [1, 60] * beta_weight )
+ np.square(1 - [1, 20] * beta_weight ) 
+ np.square(20 - [1, 30] * beta_weight)

matrix([[0.29080037]])

We then compute another F statistic between our two models

In [71]:
q = x_weight.shape[1] - 1

In [72]:
F_weight =  ((rss_weight - rss ) / q) / ( rss / (N - p - 1))
F_weight

matrix([[197.62937514]])

We can see that our subset of predictors (weight) is related to the price of the chocolate bar as the F statistic is significantly higher than 1.

## How accurate is our model?

We can measure this by looking at the $ R^2 $ statistic. An $ R^2 $ close to 1 tells us a large proportion of the variability is explained by the regression. A number closer to 0 indicates that the regression didn't explain much of the variability.

In [74]:
1 - rss/tss

matrix([[0.99918375]])